In [ ]:
import cv2
# import Libraries
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, GlobalAveragePooling2D, multiply, Lambda, Dropout,BatchNormalization, Attention
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.initializers import he_normal

In [ ]:
# Define paths
train_dir = 'D:/deep_space/MV/Dataset_split/Train'
test_dir = 'D:/deep_space/MV/Dataset_split/Test'

In [ ]:
# Load and preprocess the training data
input_shape = (256, 256, 3)
batch_size=8

# Load and preprocess the test data
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   zoom_range=[0.64, 1.0],
                                   width_shift_range=0.2,  
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.177)

'''
rotation_range=20,  # 随机旋转图像（0-180度之间）
                                   width_shift_range=0.2,  # 随机水平平移图像
                                   height_shift_range=0.2,  # 随机垂直平移图像
                                   shear_range=0.2,  # 随机错切变换
                                   zoom_range=0.2,  # 随机缩放图像
                                   horizontal_flip=True,  # 随机水平翻转图像
                                   fill_mode='nearest',  # 用于填充新创建像素的方法
                                   brightness_range=[0.5, 1.5],

'''

                                   


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    color_mode='rgb',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    color_mode='rgb',
    subset='validation'
)


In [ ]:
# Load and preprocess the test data
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    color_mode='rgb'
)

In [ ]:
# get img from generator
images, labels = next(train_generator)

# show the img
fig, axes = plt.subplots(1, batch_size, figsize=(15, 5))
for i in range(batch_size):
    axes[i].imshow(images[i])
    axes[i].set_title(f'Label: {np.argmax(labels[i])}')  # show label
    axes[i].axis('off')
plt.show()

In [ ]:
from keras.layers import Add, Activation


def residual_block(inputs, num_channels, use_1x1conv=True, strides=1):
    # Convolutional layer 1
    x = Conv2D(num_channels, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal')(inputs)
    x = Conv2D(num_channels, (3, 3), padding='same', strides=2, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    
    # Shortcut connection (identity mapping)
    if use_1x1conv:
        shortcut = MaxPooling2D(pool_size=(2,2 ), strides=(2, 2))(inputs)
        shortcut = BatchNormalization()(shortcut)
    else:
        shortcut = inputs
        
        # shortcut = MaxPooling2D(pool_size=(2,2 ), strides=(2, 2))(shortcut)
    
    # Element-wise addition
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    
    return x

In [ ]:

from keras import Input


def baseModel():
    input_tensor = Input(shape=input_shape)
    
    x = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal')(input_tensor)
    x = residual_block(x, 64)
    
    x = residual_block(x, 128)
    
    x = residual_block(x, 128)
    
    x = residual_block(x, 256)
    
    x = Flatten()(x)
    
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Uncomment the following lines if you want to include attention layer
    # attention = Attention()([x, x])
    # merged = tf.keras.layers.Concatenate(axis=-1)([x, attention])
    
    x = Dense(4, activation='softmax')(x)
    
    model = Model(inputs=input_tensor, outputs=x)
    return model



In [ ]:
model = baseModel()

In [ ]:
# model = base_model(input_shape)

In [ ]:
#computing attention weights. Each element in the input tensor x will be assigned a weight by this dense layer
# attention = Dense(1, activation='tanh')(x)

In [ ]:
# Reshape attention to have dimensions (-1, 1, 1)
# attention = Reshape((-1, 1, 1))(attention)

In [ ]:
# Apply the attention weights element-wise to the original feature representation (x) 
# attention = multiply([x, attention])

In [ ]:
# The attention weights are summed along the channel axis (-2) to produces a single attention value 
# attention = Lambda(lambda x: K.sum(x, axis=-2))(attention)

In [ ]:
# The attention values are expanded along a new axis (-1)
# attention = Lambda(lambda x: K.expand_dims(x, axis=-1))(attention)

In [ ]:
# computes the average of all the values along each channel
# attention = GlobalAveragePooling2D()(attention)

In [ ]:
# merged = Dense(4, activation='softmax')(attention)

In [ ]:
# Create the new model
# model = Model(inputs=base_model.input, outputs=merged)
# model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Display the model summary
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='sgd', 
              loss='categorical_crossentropy', 
              metrics=['accuracy']
              )

In [ ]:
# Train the model
history = model.fit(train_generator, validation_data=(val_generator), epochs=40)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(train_generator)
print('Test Accuracy:',test_accuracy)
print('Test loss:',test_loss )

In [ ]:
test_loss, test_accuracy = model.evaluate(val_generator)
print('Test Accuracy:',test_accuracy)
print('Test loss:',test_loss )

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()